In [1]:
import numpy as np
import pandas as pd
import pyfixest as pf
from IPython.display import display, FileLink
from tabout import TabOut,  DTable, ETable

# Create a multiindex dataframe with random data
row_index = pd.MultiIndex.from_tuples(
    [
        ("Gr 1", "Var 1"),
        ("Gr 1", "Var 2"),
        ("Gr 1", "Var 3"),
        ("Gr 2", "Var 4"),
        ("Gr 2", "Var 5"),
        ("Gr 3", "Var 6"),
    ]
)

col_index = pd.MultiIndex.from_product([["Hello", "There"], ["France", "US"], ["High", "Low"]])
dfa = pd.DataFrame(np.random.randn(6, 8).round(3), index=row_index, columns=col_index)


tab = TabOut(dfa, notes="Sample very new notes!", caption="The first table", tab_label="tab_sample_1")

# Generate table
# tab.make()

# Save table
tab.save(type="docx", file_name="C:/PythonProjects/tabout/docs/test_tab1", replace=True)

# Update word document with table
#tab.update_docx(file_name="C:/PythonProjects/tabout/docs/PaperTest.docx", tab_num=1, show=False)

#pf.make_table(df=df, caption="This is table 7!", notes="These are brand new notes", type="docx", 
#          file_name="C:/PythonProjects/pyfixestStore/test.docx", tab_num=7)

GT(_tbl_data=  level_0 level_1      0      1      2      3      4      5      6      7
0    Gr 1   Var 1 -1.312 -0.214 -0.022 -0.483  1.103 -2.118  1.882  0.014
1    Gr 1   Var 2  0.282 -0.891 -0.068  0.007  2.915 -0.036 -0.745 -0.452
2    Gr 1   Var 3  0.026 -0.515  2.191 -0.390  1.788 -0.807 -0.820  0.648
3    Gr 2   Var 4  0.088  0.985 -1.279 -0.636 -0.904  1.176 -0.545  0.431
4    Gr 2   Var 5  1.858 -0.378  0.312  0.094 -0.178 -1.239 -0.075  0.466
5    Gr 3   Var 6 -0.335 -1.960 -1.778 -0.055  0.417  0.679  0.262  0.912, _body=<great_tables._gt_data.Body object at 0x0000027CBFAF3950>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='High', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='Low', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CBFD356D0>, _spanners=Spanners([SpannerInfo(spanner_id='Hello', spanner_level=2, spanner_label='Hello', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None), SpannerInfo(spanner_id='There', spanner_level=2, spanner_label='There', spanner_units=None, spanner_pattern=None, vars=['4', '5', '6', '7'], built=None), SpannerInfo(spanner_id='France', spanner_level=1, spanner_label='France', spanner_units=None, spanner_pattern=None, vars=['0', '1', '4', '5'], built=None), SpannerInfo(spanner_id='US', spanner_level=1, spanner_label='US', spanner_units=None, spanner_pattern=None, vars=['2', '3', '6', '7'], built=None)]), _heading=Heading(title='The first table', subtitle=None, preheader=None), _stubhead=None, _source_notes=['Sample very new notes!'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CBFD4C470>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, categor

In [2]:
n=500
df1 = pd.DataFrame(index=range(n))
df1['ability'] = np.random.normal(100, 15, n)
df1['year'] = 1
# This is a number identifying each person in the data set
df1['persnr'] = df1.index
# Add age
df1['age'] = np.random.randint(18, 60, n)
# Add city cluster: assign each person to a city (e.g., 20 cities)
n_cities = 20
city_names = [f'City_{i+1}' for i in range(n_cities)]
df1['city'] = np.random.choice(city_names, size=n)
# Generate city-specific effects
city_effects = {city: np.random.normal(0, 2000) for city in city_names}
# Randomize three treatment groups (0, 1, 2)
df1['tgroup'] = np.random.choice([0, 1, 2], size=n, p=[1/3, 1/3, 1/3])
# Copy the DataFrame to create second year observations stored in df2
df2 = df1.copy()
df2['year'] = 2
df2['age'] = df2['age'] + 1
# Training only in the treatment group and only in year 2:
df1['training'] = 0
df2['training'] = (df2.tgroup > 0).astype(int)
# Generate DataFrame that spans both years combining the two:
# Note for each person we have two rows, one for each year
df = pd.concat([df1, df2], sort=False)
# Generate sales as before
# but add a year effect: sales increase by 4000 Euro between years 1 and 2
# and different treatment effects for each group
# and a city-specific effect
df['sales'] = (
    10000
    + (df.tgroup == 1) * 5000
    + (df.tgroup == 2) * 8000
    + df.training * 2000
    + df.ability * 100
    + df.year * 4000
    + df['city'].map(city_effects)
    + np.random.normal(0, 4000, 2 * n)
 )

labels = {"ability": "Ability (test score)",
    "sales": "Sales",
    "year": "Year",
    "age": "Age",
    "tgroup": "Treatment Group",
    "city": "City",}

df["tgroup"] = pd.Categorical(df["tgroup"], categories=[0, 1, 2], ordered=True)
df["tgroup"] = df["tgroup"].cat.rename_categories(["Control", "Treatment 1", "Treatment 2"])


In [3]:
#est=pf.feols("sales ~ ability + age + training | city", data=df)
est=pf.feols("sales ~ ability + csw(age, training) | sw0(city)", data=df)
pf.etable(est, labels=labels, drop=["age"])

GT(_tbl_data=  level_0               level_1                             0  \
0    coef  Ability (test score)       90.695*** <br> (13.663)   
1    coef              training                                 
2    coef             Intercept  2.096E+04*** <br> (1529.527)   
3      fe                  City                             -   
4   stats          Observations                          1000   
5   stats             S.E. type                           iid   
6   stats         R<sup>2</sup>                         0.044   
7   stats  R<sup>2</sup> Within                             -   

                              1                        2  \
0      105.612*** <br> (11.162)  91.862*** <br> (13.602)   
1    8156.681*** <br> (363.582)                            
2  1.727E+04*** <br> (1258.052)                            
3                             -                        x   
4                          1000                     1000   
5                           iid                 by: city   
6                         0.365                    0.192   
7                             -                    0.052   

                            3  
0     105.401*** <br> (9.908)  
1  8037.849*** <br> (321.247)  
2                              
3                           x  
4                        1000  
5                    by: city  
6                       0.502  
7                       0.416  , _body=<great_tables._gt_data.Body object at 0x0000027CC80D3650>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CC80D2C60>, _spanners=Spanners([SpannerInfo(spanner_id='Sales', spanner_level=1, spanner_label='Sales', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CC80D2120>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='

In [4]:
et1 = ETable(est, labels=labels)
et1.save(type="docx", file_name="C:/PythonProjects/tabout/docs/test_reg1", replace=True)

GT(_tbl_data=  level_0               level_1                             0  \
0    coef  Ability (test score)       90.695*** <br> (13.663)   
1    coef                   Age          16.039 <br> (17.660)   
2    coef              training                                 
3    coef             Intercept  2.096E+04*** <br> (1529.527)   
4      fe                  City                             -   
5   stats          Observations                          1000   
6   stats             S.E. type                           iid   
7   stats                    R2                         0.044   
8   stats             R2 Within                             -   

                              1                        2  \
0      105.612*** <br> (11.162)  91.862*** <br> (13.602)   
1           2.881 <br> (14.413)     23.304 <br> (17.860)   
2    8156.681*** <br> (363.582)                            
3  1.727E+04*** <br> (1258.052)                            
4                             -                        x   
5                          1000                     1000   
6                           iid                 by: city   
7                         0.365                    0.192   
8                             -                    0.052   

                            3  
0     105.401*** <br> (9.908)  
1        10.589 <br> (13.832)  
2  8037.849*** <br> (321.247)  
3                              
4                           x  
5                        1000  
6                    by: city  
7                       0.502  
8                       0.416  , _body=<great_tables._gt_data.Body object at 0x0000027CC80D0E30>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CC6782E10>, _spanners=Spanners([SpannerInfo(spanner_id='sales', spanner_level=1, spanner_label='sales', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CC6783CE0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_w

In [5]:
import statsmodels.formula.api as smf

reg=smf.ols("sales ~ ability + age + training", data=df).fit()
et2 = ETable(reg, labels=labels)
et2.save(type="docx", file_name="C:/PythonProjects/tabout/docs/test_reg3", replace=True)


GT(_tbl_data=  level_0               level_1                             0
0    coef  Ability (test score)      105.612*** <br> (11.162)
1    coef                   Age           2.881 <br> (14.413)
2    coef              training    8156.681*** <br> (363.582)
3    coef             Intercept  1.727E+04*** <br> (1258.052)
4   stats          Observations                      1000.000
5   stats             S.E. type                     nonrobust
6   stats                    R2                         0.365
7   stats               Adj. R2                         0.363, _body=<great_tables._gt_data.Body object at 0x0000027CC67D35F0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CC68932C0>, _spanners=Spanners([SpannerInfo(spanner_id='sales', spanner_level=1, spanner_label='sales', spanner_units=None, spanner_pattern=None, vars=['0'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CC6893DD0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='

In [6]:
et2

<tabout.tabout.TabOut.make.<locals>.DualOutput at 0x27cc681aed0>

In [7]:
dt1=DTable(df, vars=["ability","age"], bycol=["tgroup"], stats=["mean_newline_std"], counts_row_below=True, hide_stats=True, labels=labels)
dt1

<tabout.tabout.TabOut.make.<locals>.DualOutput at 0x27cc6823560>

In [8]:
dt1.save(type="tex", file_name="C:/PythonProjects/tabout/docs/test_tab2", replace=True)

GT(_tbl_data=  level_0               level_1            Control       Treatment 1  \
0   stats  Ability (test score)  102.99<br>(15.41)  97.88<br>(14.94)   
1   stats                   Age   39.27<br>(11.64)  39.27<br>(11.81)   
2    nobs                     N                336               334   

         Treatment 2  
0  100.61<br>(15.31)  
1   40.14<br>(12.19)  
2                330  , _body=<great_tables._gt_data.Body object at 0x0000027CC6B00560>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='Control', type=<ColInfoTypeEnum.default: 1>, column_label='Control', column_align='center', column_width=None), ColInfo(var='Treatment 1', type=<ColInfoTypeEnum.default: 1>, column_label='Treatment 1', column_align='center', column_width=None), ColInfo(var='Treatment 2', type=<ColInfoTypeEnum.default: 1>, column_label='Treatment 2', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CC6B006B0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Note: Displayed statistics are Mean (Std. Dev.).'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CC6B02DE0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', va

In [9]:
dt1.update_docx(file_name="C:/PythonProjects/tabout/docs/PaperTest.docx", tab_num=1, show=False) 

In [10]:
dt=DTable(df, vars=["ability","age"], bycol=["tgroup"], stats=["mean_newline_std"], counts_row_below=True, labels=labels, hide_stats=True)

dt.update_docx(file_name="C:/PythonProjects/tabout/docs/PaperTest.docx", tab_num=1, show=True)

GT(_tbl_data=  level_0               level_1            Control       Treatment 1  \
0   stats  Ability (test score)  102.99<br>(15.41)  97.88<br>(14.94)   
1   stats                   Age   39.27<br>(11.64)  39.27<br>(11.81)   
2    nobs                     N                336               334   

         Treatment 2  
0  100.61<br>(15.31)  
1   40.14<br>(12.19)  
2                330  , _body=<great_tables._gt_data.Body object at 0x0000027CC6968E90>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='Control', type=<ColInfoTypeEnum.default: 1>, column_label='Control', column_align='center', column_width=None), ColInfo(var='Treatment 1', type=<ColInfoTypeEnum.default: 1>, column_label='Treatment 1', column_align='center', column_width=None), ColInfo(var='Treatment 2', type=<ColInfoTypeEnum.default: 1>, column_label='Treatment 2', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000027CC69695E0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Note: Displayed statistics are Mean (Std. Dev.).'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000027CC696BC20>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', va

In [11]:
# # Create a simple index dataframe with random data
# simple_index = ['Var 1', 'Var 2', 'Var 3', 'Var 4', 'Var 5', 'Var 6']
# simple_col_index = ['Hello', 'There', 'France', 'US', 'High', 'Low', 'Extra1', 'Extra2']
# df = pd.DataFrame(np.random.randn(6, 8).round(3), index=simple_index, columns=simple_col_index)
# tab_out = TabOut(df, notes="Sample notes", caption="Sample Table 1", tab_label="tab_sample")

# # Generate tables in different formats
# ##tab_out.make(types=["gt", "docx"], path="C:/PythonProjects/pyregtable/")

